In [10]:
import os
import sys
sys.path.insert(0,os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+'/ETA_Predictions')

import numpy as np
import scipy as sp

from collections import OrderedDict
from FP_Utilities import Read_E_Bins,Build_Nagy_Weighted_FPs,Build_Spline_Weighted_FPs, Read_FPs,Read_Fission_Spectrum

In [44]:
# User Inputs
#---------------------------------------------------------------------------------------#
fps=13  #Number of fission product measurements to use = number of bins
zaid=[92235, 92238]  # Isotopes to use, in order of lowest to highest; Currently terrible coding that limits len to 2
f=[0.978531749685,0.0214622349078]  # Fission fraction. Match order of zaids - information can be obained from model or fission split

nagy_data_fname='ETA_Nagy_fy_HEU_{}.csv'.format(fps)
bins_fname = 'Bins_{}.csv'.format(fps)   # Bin structure input file

if 92235 in zaid:
    fname_235='235_data_{}.xlsx'.format(fps)   # 235 energy dependent data - must contain same isotopes as 238      
    fiss_235_fname = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+"/ETA_Predictions/E_fiss_235.csv"   # Fission spectra input file
if 92238 in zaid:
    fname_238='238_data_{}.xlsx'.format(fps)   # 238 energy dependent data - must contain same isotopes as 235
    fiss_238_fname = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+"/ETA_Predictions/E_fiss_238.csv"   # Fission spectra input file
#---------------------------------------------------------------------------------------#  

In [45]:
# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(bins_fname)
print "Number of bins:",len(bins)
print "Energy bin structure:",bins

# Import y_A
(y_A,y_err)=np.asarray(Read_FPs(nagy_data_fname))
#print len(y_A),y_A
#print len(y_err),y_err

# Calculate FP data 
if 92235 in zaid: 
    pred_y=Build_Nagy_Weighted_FPs(fname_235,bins,f[0],np.ones_like(bins))
    pred_y=OrderedDict(sorted(pred_y.items()))
if 92238 in zaid and len(zaid)==1:
    pred_y=Build_Nagy_Weighted_FPs(fname_238,bins,f[0],np.ones_like(bins))
    pred_y=OrderedDict(sorted(pred_y.items()))
if 92238 in zaid and len(zaid)>1:
    pred_y2=Build_Nagy_Weighted_FPs(fname_238,bins,f[1],np.ones_like(bins))
    pred_y2=OrderedDict(sorted(pred_y2.items()))
    for A in pred_y2.keys():
        for i in range(0,len(pred_y[A])):
            pred_y[A][i]=pred_y[A][i]+pred_y2[A][i]

# Convert dictionary to list then numpy array
y_e=[]
for A in pred_y.keys():
    y_e.append(pred_y[A])
y_e=np.asarray(y_e).reshape((fps, fps))

#print pred_y
#print len(y_e),y_e    

Number of bins: 13
Energy bin structure: [0.25, 0.75, 1.25, 1.75, 2.5, 3.5, 5.0, 7.5, 10.5, 12.5, 13.5, 14.5, 15.5]


In [13]:
###################   Solve a system of linear equations  ##########################

# Solve for phi_f
phi_f=np.linalg.solve(y_e,y_A)
print "\nMethod #1:",sum(phi_f),phi_f
phi_f=np.linalg.lstsq(y_e,y_A)[0]
print "Method #2:",sum(phi_f),phi_f
phi_f=sp.optimize.nnls(y_e,y_A)
print "Method #3:",np.sum(phi_f[0]),phi_f

# Check Answer
print "\nCheck Answer:",np.allclose(np.dot(y_e, phi_f[0]), y_A)

# Calculate Y[A] from phi_f found
print "\nUsing phi_f, Y[A]=",np.dot(y_e, phi_f[0])

# Calculate Y[A] using actual phi_f
if fps==3:
    print "\nUsing phi_f from MCNP, Y[a]=",np.dot(y_e, np.array([0.712,0.0901,0.198]))
elif fps==5:
    print "\nUsing phi_f from MCNP, Y[a]=",np.dot(y_e, np.array([0.619,0.0931,0.0695,0.0301,0.188]))
elif fps==8:
    print "\nUsing phi_f from MCNP, Y[a]=",np.dot(y_e, np.array([0.447,0.229,0.0792,0.0102,0.0145,0.0131,0.017,0.188]))


Method #1: -0.3837890625 [  1.48705246e+13  -4.25743584e+13   1.88374637e+13   3.49593149e+13
  -3.57581459e+13   1.04607855e+13  -7.72930616e+11  -1.15102198e+12
   7.72601908e+12  -2.98501584e+13   4.05461955e+13  -2.14497437e+13
   4.15605575e+12]
Method #2: 3.88682556152 [  1.04239459e+11  -2.76656410e+11   9.58967893e+10   2.37409245e+11
  -2.15451579e+11   5.86685187e+10  -4.10664455e+09   5.85831297e+09
  -5.83142292e+10   2.80672881e+11  -4.20229249e+11   2.43236510e+11
  -5.12236033e+10]
Method #3: 1.00019493398 (array([ 0.41752283,  0.        ,  0.        ,  0.        ,  0.35229676,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.23037534,  0.        ,  0.        ]), 0.06511158778335521)

Check Answer: False

Using phi_f, Y[A]= [ 6.23247031  5.7461103   5.65159017  0.2522242   0.17918427  0.2523906
  6.38907944  5.69432824  5.48239815  2.11501668  0.47240357  0.17662987
  0.02660705]


In [14]:
################ Solve a Chi-Squared Minimization of system of linear equations  ################
from scipy.optimize import minimize

# Chi-squared minimization function for an arbitrary number of energy groups for one isotope
def chisqfunc(phi):
    phi=abs(phi/np.sum(phi))
    fy_model = np.dot(phi,np.transpose(y_e))
    #print fy_model
    chisq = np.sum(((y_A - fy_model)/y_err)**2)
    return chisq

In [15]:
# Perform minimization and output results
fs0 = np.ones_like(bins)/float(len(bins))
fiss_split =  minimize(chisqfunc, fs0, method='nelder-mead', \
                       options={'xtol': 1e-6, 'disp': True, 'maxiter': 10000,'maxfev':100000})
print fiss_split
print abs(fiss_split.x)
print np.sum(abs(fiss_split.x))
print abs(fiss_split.x)/np.sum(abs(fiss_split.x))
print np.sum(abs(fiss_split.x)/np.sum(abs(fiss_split.x)))

Optimization terminated successfully.
         Current function value: 0.049980
         Iterations: 1928
         Function evaluations: 2798
  status: 0
    nfev: 2798
 success: True
     fun: 0.0499804654788049
       x: array([  9.34973309e-02,   3.04121487e-01,   5.97064206e-02,
         1.32727129e-01,   3.63469081e-02,   5.20969362e-02,
         4.01247581e-03,   4.08206548e-02,   1.39608610e-09,
         8.47766329e-03,   5.40234325e-02,   1.07684539e-01,
         3.60732713e-03])
 message: 'Optimization terminated successfully.'
     nit: 1928
[  9.34973309e-02   3.04121487e-01   5.97064206e-02   1.32727129e-01
   3.63469081e-02   5.20969362e-02   4.01247581e-03   4.08206548e-02
   1.39608610e-09   8.47766329e-03   5.40234325e-02   1.07684539e-01
   3.60732713e-03]
0.897122305836
[  1.04219158e-01   3.38996684e-01   6.65532673e-02   1.47947642e-01
   4.05149976e-02   5.80711636e-02   4.47260734e-03   4.55017722e-02
   1.55618257e-09   9.44984116e-03   6.02185813e-02   1.2003328

In [16]:
##############  CS minimization of system of linear equations #####################
import os
import sys
sys.path.insert(0,'/home/pyne-user/Dropbox/UCB/Research/ETAs/META-CODE/Benchmark_Code')

import numpy as np
import FissionAnalysis_Obj_Functions as of
import OptiPlot as op
import CuckooSearch as cs
import CSLibrary as csl
import math as m
from operator import attrgetter

# Select optimization problem type and associated parameters
#opt_funct=of.LinEq5_Obj
#opt_funct=of.LinEq8_Obj
opt_funct=of.LinEq13_Obj

# Set optimization settings
opt_params=of.Get_Params(opt_funct,'CS',dimension=4)
S=csl.Settings(optimal_fitness=opt_params.o)

# Initialize variables
max_iter=3  #Number of algorithm iterations
eval_interval=500 # Fuction eval interval at which the fitness is sampled.  
history=[]   #List that contains the final timeline results from each optimization run 
# History of fitness vs function evals  
feval_history=np.array([[i*eval_interval,0.0,0.0,0] for i in \
                            range(int(S.em/eval_interval)+1)])     #[Feval, Fit avg, Fit std, counter]

for i in range(0,max_iter,1):
    print "Run #%d" %i
    S.d=False
    
    YCS_on=False
    CS_on=True

    #YCS
    if YCS_on:
        (timeline)=ycs.CS(opt_funct,opt_params.lb,opt_params.ub,S)

    # CS
    if CS_on:
        S.c=25
        S.pa=0.25
        S.pt=1.0/S.c
        S.s='lhc'
        S.sl=100
        S.sf=100
        S.gm=400
        (timeline)=cs.CS(opt_funct,opt_params.lb,opt_params.ub,S)
    
    # Ouput problem outputs to user
    debug=False
    if debug==True:
        print "The best design is", timeline[-1].d," with a fitness of ", timeline[-1].f 
        print "The number of generations was %d with %f objective function evals." %(timeline[-1].g,timeline[-1].e)
    
    # Save final timeline data for future processing
    mingen = min(timeline,key=attrgetter('f'))
    history.append(csl.Event(mingen.g,mingen.e,mingen.f,mingen.d))

    # Update Fitness vs Feval History
    k=1 
    
    # Compute the average and standard deviation using a recurrence relation
    for j in range(0,len(feval_history),1):
        while timeline[k].e < feval_history[j,0] and k+1<len(timeline):  
            k+=1
        if k+1 == len(timeline) and timeline[k].e < feval_history[j,0]:
            # Initialize the array on the first run
            if feval_history[j,3]==0:
                feval_history[j,1]=timeline[k].f
                feval_history[j,2]=0.0
                feval_history[j,3]+=1
            else:
                old_mean=feval_history[j,1]
                feval_history[j,3]+=1
                feval_history[j,1]=feval_history[j,1]+(timeline[k].f-feval_history[j,1])/feval_history[j,3]
                feval_history[j,2]=feval_history[j,2]+(timeline[k].f-old_mean)*(timeline[k].f-feval_history[j,1])                
            break 
        else:
            # Initialize the array on the first run
            if feval_history[j,3]==0:
                feval_history[j,1]=timeline[k-1].f
                feval_history[j,2]=0.0
                feval_history[j,3]+=1
            else:    
                old_mean=feval_history[j,1]
                feval_history[j,3]+=1
                feval_history[j,1]=feval_history[j,1]+(timeline[k-1].f-feval_history[j,1])/feval_history[j,3]
                feval_history[j,2]=feval_history[j,2]+(timeline[k-1].f-old_mean)*(timeline[k-1].f-feval_history[j,1])
            
#    op.Plot_Feval_Hist(data=feval_history)
    
# Calculate averages and standard deviations
tmp=[]
for i in range(len(history[-1].d)):
    tmp.append(sum(c.d[i] for c in history)/float(len(history)))
averages=csl.Event(sum(c.g for c in history)/float(len(history)),sum(c.e for c in history)/float(len(history)), \
        sum(c.f for c in history)/float(len(history)),tmp)

tmp=[]
if max_iter >1:
    for i in range(len(history[-1].d)):
        tmp.append(m.sqrt(sum([(c.d[i] - averages.d[i])**2 for c in history])/(len(history) - 1)))
    std_dev=csl.Event(m.sqrt(sum([(c.g - averages.g)**2 for c in history])/(len(history) - 1)),\
                  m.sqrt(sum([(c.e - averages.e)**2 for c in history])/(len(history) - 1)), \
                  m.sqrt(sum([(c.f - averages.f)**2 for c in history])/(len(history) - 1)),tmp)
else:
    tmp=np.zeros(len(history[-1].d))
    std_dev=csl.Event(0,0,0.0,tmp)
    
# Trim empty feval histories 
if feval_history[-1,3]==0:
    feval_history=np.array([feval_history[tmp,:] for tmp in \
                       range(len(feval_history[0:(np.argmin(feval_history[:,3]))]))])
# Compute relative error (%Diff) for feval histories
if S.of==0.0:
    feval_history[:,1]=feval_history[:,1]*100
else:
    feval_history[:,1]=abs((feval_history[:,1]-S.of)/S.of*100)
# Compute standard deviation for feval histories
for i in range(0,len(feval_history)):
    if feval_history[i,2]!=0.0:
        feval_history[i,2]=np.sqrt(feval_history[i,2]/(feval_history[i,3]-1))*100

# Output average results to the user
debug=True
if debug==True:
    print "\nThe Average Optimized Solution:"
    print "================================"
    print "Design:"
    for i in range(len(averages.d)):
        print "   var #%d: %4.6f $\mypm$ %4.5f " %(i+1,averages.d[i],std_dev.d[i])
    print "Fitness: %4.6f $\mypm$ %4.5f " %(averages.f,std_dev.f)
    print "Funct Evals: %d $\mypm$ %d " %(averages.e,std_dev.e)
    print "Generations: %3.1f $\mypm$ %3.1f "  %(averages.g,std_dev.g)
    if S.of==0.0:
        print "The performance metric is %4.1f" %(averages.f*(averages.e+3*std_dev.e))
    else:
        
        print "The performance metric is %4.1f" %(abs((averages.f-S.of)/S.of)*(averages.e+3*std_dev.e))

#Determine the best values obtained
best=min(history,key=attrgetter('f'))

# Output best result to the user
debug=True
if debug==True:
    print "\nThe Best Optimized Solution:"
    print "================================"
    print "Design:"
    for i in range(len(averages.d)):
        print "   var #%d: %4.6f " %(i+1,best.d[i])
    print "Fitness: %4.6f " %best.f
    print "Funct Evals: %d " %best.e
    print "Generations: %d "  %best.g
    if S.of==0.0:
        print "The performance metric is %4.1f" %(best.f*best.e)
    else:
        print "The performance metric is %4.1f" %(abs((best.f-S.of)/S.of)*best.e)
        
#Plot the optimization process
op.Plot_Vars(timeline,low_bounds=opt_params.lb,up_bounds=opt_params.ub,title=opt_params.pt,label=opt_params.l)
#fevals=[tmp.e for tmp in history]
#if max_iter >1:
#    op.Plot_Hist(fevals,title=opt_params.ht)
#op.Plot_Feval_Hist(data=feval_history)

Run #0
Generational Stall 291 190
Program execution time was 178.248934.
Run #1
Program execution time was 245.672587.
Run #2
Fitness Convergence
Program execution time was 218.869224.

The Average Optimized Solution:
Design:
   var #1: 0.197459 $\mypm$ 0.23568 
   var #2: 0.292806 $\mypm$ 0.25795 
   var #3: 0.074202 $\mypm$ 0.10377 
   var #4: 0.079052 $\mypm$ 0.03069 
   var #5: 0.042204 $\mypm$ 0.03249 
   var #6: 0.046005 $\mypm$ 0.04123 
   var #7: 0.016671 $\mypm$ 0.01473 
   var #8: 0.039197 $\mypm$ 0.03367 
   var #9: 0.014919 $\mypm$ 0.01121 
   var #10: 0.063670 $\mypm$ 0.05226 
   var #11: 0.086192 $\mypm$ 0.08442 
   var #12: 0.040036 $\mypm$ 0.04937 
   var #13: 0.008430 $\mypm$ 0.00834 
Fitness: 0.017103 $\mypm$ 0.01462 
Funct Evals: 4518 $\mypm$ 780 
Generations: 335.7 $\mypm$ 57.8 
The performance metric is 117.3

The Best Optimized Solution:
Design:
   var #1: 0.134010 
   var #2: 0.283253 
   var #3: 0.193656 
   var #4: 0.063893 
   var #5: 0.055466 
   var #6: 0.00

In [46]:
# Determine the MCNP modeled group fission rates
if 92235 in zaid:
    (fiss_235_e,fiss_235)=Read_Fission_Spectrum(fiss_235_fname)
    sum_fiss=np.sum(fiss_235)
    fiss=fiss_235/sum_fiss*f[0]
if 92238 in zaid:
    (fiss_238_e,fiss_238)=Read_Fission_Spectrum(fiss_238_fname)
    sum_fiss=np.sum(fiss_238)
    if len(zaid)==1:
        fiss=fiss_238/sum_fiss*f[len(zaid)-1]
    if len(zaid)>1:
        fiss+=fiss_238/sum_fiss*f[len(zaid)-1]

# Sum within a group bound
j=0
grp_fiss=np.zeros_like(upper_bins)
for i in range(0,len(fiss)):
    if fiss_235_e[i]<upper_bins[j]:
        grp_fiss[j]+=fiss[i]
    elif fiss_235_e[i] > upper_bins[j]:
        if j!= len(upper_bins)-1:
            grp_fiss[j]+=(upper_bins[j]-fiss_235_e[i-1])/(fiss_235_e[i]-fiss_235_e[i-1])*fiss[i]
            grp_fiss[j+1]+=(fiss_235_e[i]-upper_bins[j])/(fiss_235_e[i]-fiss_235_e[i-1])*fiss[i]
            j+=1
        else:
            grp_fiss[j]+=fiss[i]

In [47]:
print "Objective:"
print grp_fiss
print "\nSystem of Linear Equations:"
print phi_f[0]/sum(phi_f[0])
print "\nMinimize Chi-Squared Optimization:"
print abs(fiss_split.x)/np.sum(abs(fiss_split.x))
print "\nCS Chi-Squared Optimization"
print sum(averages.d/sum(averages.d)),averages.d/sum(averages.d)

Objective:
[  2.78626339e-01   1.69666857e-01   1.08397191e-01   7.01369449e-02
   7.60820131e-02   2.90011516e-02   2.46652049e-02   2.43846569e-02
   2.13484560e-02   1.72315371e-02   9.78446112e-02   8.25634581e-02
   4.55633507e-05]

System of Linear Equations:
[ 0.41744145  0.          0.          0.          0.3522281   0.          0.
  0.          0.          0.          0.23033044  0.          0.        ]

Minimize Chi-Squared Optimization:
[  1.04219158e-01   3.38996684e-01   6.65532673e-02   1.47947642e-01
   4.05149976e-02   5.80711636e-02   4.47260734e-03   4.55017722e-02
   1.55618257e-09   9.44984116e-03   6.02185813e-02   1.20033286e-01
   4.02099815e-03]

CS Chi-Squared Optimization
1.0 [ 0.19729262  0.29255894  0.07413989  0.07898514  0.04216872  0.04596619
  0.01665652  0.03916369  0.01490652  0.06361671  0.08611953  0.04000268
  0.00842286]
